In [1]:
import os
os.chdir(os.pardir)

In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999

import PySAM.Pvsamv1 as pvs
import PySAM.Pvwattsv7 as pvw
import PySAM.Battwatts as batt
import PySAM.StandAloneBattery as stbatt
import PySAM.Grid as gr
import PySAM.Utilityrate5 as ur
import PySAM.Cashloan as cl
import PySAM.Merchantplant as mp
from PySAM.BatteryTools import size_li_ion_battery, battery_model_sizing

import vapor
import vapor.config as config

load = vapor.BuildingLoad()
load.load()

resource_fp = 'data/PySAM Downloaded Weather Files/-87.031_42.192_psm3_60_tmy.csv'
cambium_df = vapor.datafetcher.load_cambium_data(aggregate_region='census_reg', scenario='StdScen19_Mid_Case')

desired_system_capacity = 1000
desired_inverter_kw = desired_system_capacity / 1.2
modules_on_string = 12
module_capacity = 0.31
string_capacity = modules_on_string * module_capacity
n_string = int(desired_system_capacity / string_capacity)
inverter_capacity = 791

gen = pvs.default('FlatPlatePVSingleOwner')
gen.SolarResource.solar_resource_file = resource_fp
gen.SystemDesign.subarray1_nstrings = n_string
gen.SystemDesign.inverter_count = int(desired_inverter_kw / inverter_capacity)
gen.Lifetime.dc_degradation = [0.5]
gen.SystemDesign.system_capacity = n_string * string_capacity
gen.Lifetime.system_use_lifetime_output = 1
gen.Lifetime.analysis_period = config.SYSTEM_LIFETIME
gen.execute()


batt = stbatt.from_existing(gen) #Simualtion, Lifetime, BatterySystem, SystemOutput, Load, BatteryCell, Inverter, Losses, batteryDispatch, ElectricityRates, FuelCell, PriceSignal
batt.BatterySystem.en_batt = 1
batt.BatterySystem.batt_meter_position = 0 # 0: BTM, 1: FTM
batt.BatterySystem.batt_ac_or_dc = 1 #0: dc, 1: ac
batt.BatterySystem.batt_dc_ac_efficiency = 96
batt.BatterySystem.batt_dc_dc_efficiency = 98
batt.BatterySystem.batt_ac_dc_efficiency = 98
batt.BatterySystem.batt_current_choice = 1
batt.BatterySystem.batt_replacement_capacity = 50
batt.BatterySystem.batt_replacement_option = 1
batt.BatterySystem.batt_surface_area = (1.586 ** 2) * 6
batt.BatterySystem.batt_mass = 10133.271
batt.BatterySystem.batt_inverter_efficiency_cutoff = 90
batt.BatteryDispatch.batt_dispatch_choice = 0 #0: peak shaving look ahead, 1: peak shaving look behind
batt.BatteryDispatch.batt_dispatch_auto_can_clipcharge = 1
batt.BatteryDispatch.batt_dispatch_auto_can_charge = 1
batt.BatteryDispatch.batt_dispatch_auto_can_gridcharge = 0

# --- Size load to match system aep ---
load.scale(gen.Outputs.annual_energy / 1000)
load_profile = load.as_array()
batt.Load.load = load_profile
batt.BatteryCell.batt_chem = 1
batt.BatteryCell.batt_Vnom_default = 3.6
batt.BatteryCell.batt_Vfull = 4.1
batt.BatteryCell.batt_Vexp = 4.05
batt.BatteryCell.batt_Vnom = 3.4
batt.BatteryCell.batt_Qfull = 2.25
batt.BatteryCell.batt_Qfull_flow = 3996
batt.BatteryCell.batt_Qexp = 0.040049999952316284 
batt.BatteryCell.batt_Qnom = 2.0002501010894775
batt.BatteryCell.batt_C_rate = 0.2
batt.BatteryCell.batt_resistance = 0.2
batt.BatteryCell.batt_initial_SOC = 50
batt.BatteryCell.batt_minimum_SOC = 15
batt.BatteryCell.batt_maximum_SOC = 95
batt.BatteryCell.batt_minimum_modetime = 10
batt.BatteryCell.batt_calendar_choice = 0
batt.BatteryCell.batt_lifetime_matrix = [[20, 0, 100], [20, 5000, 80], [20, 10000, 60], [80, 0, 100], [80, 1000, 80], [80, 2000, 60]];
batt.BatteryCell.batt_calendar_lifetime_matrix = [[0, 100], [3650, 80], [7300, 50]];
batt.BatteryCell.batt_calendar_q0 = 1.02
batt.BatteryCell.batt_calendar_a = 0.00266
batt.BatteryCell.batt_calendar_b = -7280
batt.BatteryCell.batt_calendar_c = 930
batt.BatteryCell.batt_voltage_matrix = [[0, 0]]
batt.BatteryCell.cap_vs_temp = [[-10, 60], [0, 80], [25, 100], [40, 100]]
batt.BatteryCell.batt_Cp = 1004
batt.BatteryCell.batt_h_to_ambient = 5
batt.BatteryCell.batt_room_temperature_celsius = np.full(8760, fill_value=20)
fitted_batt = battery_model_sizing(batt, desired_power=1000,
                     desired_capacity=4000,
                     desired_voltage=500)
batt.execute()


financial = mp.from_existing(gen) #FinancialParamters, SystemCosts, TaxCreditIncentives, Depreciation, PaymentIncentives, Revenue, batterySystem, SystemOutput, UtilityBill, Lifetime, FuelCell, Capacity Payments, GridLimits
financial.FinancialParameters.debt_option = config.DEBT_OPTION
financial.FinancialParameters.debt_percent = config.DEBT_PERCENT * 100
financial.FinancialParameters.inflation_rate = config.INFLATION * 100
financial.FinancialParameters.dscr = config.DSCR
financial.FinancialParameters.real_discount_rate = config.DISCOUNT_RATE * 100
financial.FinancialParameters.term_int_rate = config.INTEREST_RATE * 100
financial.FinancialParameters.term_tenor = config.SYSTEM_LIFETIME
financial.FinancialParameters.insurance_rate = 0
financial.FinancialParameters.federal_tax_rate = [21]
financial.FinancialParameters.state_tax_rate = [7]
financial.FinancialParameters.property_tax_rate = 0
financial.FinancialParameters.prop_tax_cost_assessed_percent = 100
financial.FinancialParameters.prop_tax_assessed_decline = 0
financial.Depreciation.depr_custom_schedule = [0]
financial.SystemCosts.om_fixed = [0]
financial.SystemCosts.om_fixed_escal = 0
financial.SystemCosts.om_production = [0]
financial.SystemCosts.om_production_escal = 0
financial.SystemCosts.om_capacity_escal = 0
financial.SystemCosts.om_fuel_cost = [0]
financial.SystemCosts.om_fuel_cost_escal = 0
financial.SystemCosts.om_replacement_cost_escal = 0
financial.SystemOutput.degradation = [0.5]
financial.SystemOutput.system_pre_curtailment_kwac = gen.Outputs.gen


# --- Consider all generation as revenue at the market price ---
# We are valuing the output to both the grid (and the developer) as the cambium market price. 
# this values self-consumption at the hourly market clearing price, same as exports. 
# however, battery charging (which can not be from the grid) and discharging is optimized around load
# using a peak shaving algorithm. 
# this way, we are simulating a system designed to first, provide the maximum environmental benefits,
# while being valued using real-time costs of energy.
market = vapor.Cambium(cambium_df, 'census_reg','ENC', resource_fp)
market.clean()
market_profile = market.calc_cambium_lifetime_rev_tuple(gen.Outputs.gen, 'cambium_total_value')
financial.Revenue.mp_enable_ancserv1 = 0
financial.Revenue.mp_enable_ancserv2 = 0
financial.Revenue.mp_enable_ancserv3 = 0
financial.Revenue.mp_enable_ancserv4 = 0
financial.Revenue.mp_enable_energy_market_revenue = 1
financial.Revenue.mp_energy_market_revenue = market_profile
financial.Revenue.mp_ancserv1_revenue = ((0, 0),(0, 0))
financial.Revenue.mp_ancserv2_revenue = ((0, 0),(0, 0))
financial.Revenue.mp_ancserv3_revenue = ((0, 0),(0, 0))
financial.Revenue.mp_ancserv4_revenue = ((0, 0),(0, 0))
financial.CapacityPayments.cp_capacity_payment_type = 0
financial.CapacityPayments.cp_capacity_payment_amount = [0]
financial.CapacityPayments.cp_capacity_credit_percent = [0]
financial.CapacityPayments.cp_capacity_payment_esc = 0
financial.CapacityPayments.cp_system_nameplate = financial.SystemOutput.system_capacity
financial.CapacityPayments.cp_battery_nameplate = batt.BatterySystem.batt_power_discharge_max_kwac


# --- Calculate System Costs --- 
cost = vapor.SystemCost()
cost.load()
capex_kW = cost.get_capex(region='CO', tech='pv')
system_cost = gen.SystemDesign.system_capacity * capex_kW
cost_per_inverter = 60 * 0.07 * 1000
marginal_inverter_cost = 1 * cost_per_inverter
system_cost += marginal_inverter_cost
batt_kW = cost.get_capex(region='CO', tech='battery', duration=4)
system_cost += (batt_kW * batt.BatterySystem.batt_power_discharge_max_kwac)
financial.SystemCosts.total_installed_cost = system_cost
financial.FinancialParameters.construction_financing_cost = system_cost * 0.009

# --- Calcualte O&M Costs ---
om_cost = cost.get_om(region='CO', tech='pv')
financial.SystemCosts.om_capacity = [om_cost]
financial.execute()

SyntaxError: invalid syntax (&lt;ipython-input-2-0bf9ad971a9e&gt;, line 20)

In [1]:
import matplotlib.pyplot as plt

ts = pd.DataFrame({
    'load':load_profile,
    'grid_to_load':batt.Outputs.grid_to_load[0:8760],
    'batt_to_load':batt.Outputs.batt_to_load[0:8760],
    'pv_to_batt':batt.Outputs.pv_to_batt[0:8760],
    'pv_to_grid':batt.Outputs.pv_to_grid[0:8760],
    'pv_to_load':batt.Outputs.pv_to_load[0:8760],
    'grid_to_batt':batt.Outputs.grid_to_batt[0:8760],
    'gen':gen.Outputs.gen[0:8760]
})

fig, ax = plt.subplots()
ts[24*180:(24*180)+(24*5)].plot(ax=ax)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

NameError: name &#39;pd&#39; is not defined

In [12]:
ts

,load,grid_to_load,batt_to_load,pv_to_batt,pv_to_grid,pv_to_load,grid_to_batt,gen
0,103.226645,12.026541,91.200104,0.0,0.0,0.0,0.0,90.966794
1,102.246090,12.026469,90.219621,0.0,0.0,0.0,0.0,89.986311
2,106.091659,12.026470,94.065189,0.0,0.0,0.0,0.0,93.831879
3,103.458191,12.026470,91.431721,0.0,0.0,0.0,0.0,91.198411
4,107.069108,12.026472,95.042636,0.0,0.0,0.0,0.0,94.809326
...,...,...,...,...,...,...,...,...
8755,93.925791,93.925791,0.000000,0.0,0.0,0.0,0.0,-0.233310
8756,94.193211,94.193211,0.000000,0.0,0.0,0.0,0.0,-0.233310
8757,94.669448,94.669448,0.000000,0.0,0.0,0.0,0.0,-0.233310
8758,94.628187,94.628187,0.000000,0.0,0.0,0.0,0.0,-0.233310


In [8]:
ts.sum()

load            4.906340e+06
grid_to_load    1.753770e+06
batt_to_load    3.205773e+05
pv_to_batt      3.549890e+05
pv_to_grid      1.719358e+06
pv_to_load      2.831993e+06
grid_to_batt    0.000000e+00
gen             4.871931e+06
dtype: float64

In [9]:
gen.Outputs.annual_energy

4871931.147249881

In [4]:
def _set_num_turbines_in_row(n_turbines, rotor_diameter=77, spacing=None, angle_deg=0):
    xcoords = []
    ycoords = []

    row_spacing = max(spacing, rotor_diameter * 3)

    dx = row_spacing * np.cos(np.radians(angle_deg))
    dy = row_spacing * np.sin(np.radians(angle_deg))
    x0 = 0
    y0 = 0

    for i in range(n_turbines):
        turb = Point((x0 + i * dx, y0 + i * dy))
        xcoords.append(turb.x)
        ycoords.append(turb.y)

    return xcoords, ycoords

def _single_year_to_multiyear_gen(gen, degradation, analysis_period):
    gen_profile = gen.Outputs.gen
    assert len(gen_profile) == 8760
    # --- apply degradation --- 
    years = []
    for y in range(0, int(analysis_period)):
        gen_year = np.array(gen_profile)
        gen_year = gen_year * ((1 - degradation) ** y)
        years.append(gen_year)

    # --- concatenate ---
    out = np.concatenate(years).ravel().tolist()
    return out

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
pd.options.display.max_rows = 999

import PySAM.Windbos as wbos
import PySAM.Pvsamv1 as pvs
import PySAM.Pvwattsv7 as pvw
import PySAM.Battwatts as batt
import PySAM.Windpower as wp
import PySAM.StandAloneBattery as stbatt
import PySAM.Grid as gr
import PySAM.Utilityrate5 as ur
import PySAM.Cashloan as cl
import PySAM.Merchantplant as mp
from PySAM.BatteryTools import size_li_ion_battery, battery_model_sizing

import vapor
import vapor.config as config

load = vapor.BuildingLoad()
load.load()

resource_fp = 'data/PySAM Downloaded Weather Files/-87.031_42.192_psm3_60_tmy.csv'
cambium_df = vapor.datafetcher.load_cambium_data(aggregate_region='inter', scenario='StdScen19_Mid_Case')

desired_system_capacity = 1500

gen = wp.default('WindPowerSingleOwner') #Resource, Turbine, Farm, Losses, Uncertainty, AdjustmentFactors
gen.Resource.wind_resource_filename = 'data/PySAM Downloaded Weather Files/-99.347_30.927_wtk_60_2012.srw'
gen.Resource.wind_resource_model_choice = 0
gen.Resource.weibull_reference_height = 50
gen.Resource.weibull_k_factor = 2
gen.Resource.weibull_wind_speed = 7.25
gen.Turbine.wind_resource_shear = 0.14
gen.Turbine.wind_turbine_rotor_diameter = 77
gen.Turbine.wind_turbine_powercurve_windspeeds =[ 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5, 5.25, 5.5, 5.75, 6, 6.25, 6.5, 6.75, 7, 7.25, 7.5, 7.75, 8, 8.25, 8.5, 8.75, 9, 9.25, 9.5, 9.75, 10, 10.25, 10.5, 10.75, 11, 11.25, 11.5, 11.75, 12, 12.25, 12.5, 12.75, 13, 13.25, 13.5, 13.75, 14, 14.25, 14.5, 14.75, 15, 15.25, 15.5, 15.75, 16, 16.25, 16.5, 16.75, 17, 17.25, 17.5, 17.75, 18, 18.25, 18.5, 18.75, 19, 19.25, 19.5, 19.75, 20, 20.25, 20.5, 20.75, 21, 21.25, 21.5, 21.75, 22, 22.25, 22.5, 22.75, 23, 23.25, 23.5, 23.75, 24, 24.25, 24.5, 24.75, 25, 25.25, 25.5, 25.75, 26, 26.25, 26.5, 26.75, 27, 27.25, 27.5, 27.75, 28, 28.25, 28.5, 28.75, 29, 29.25, 29.5, 29.75, 30, 30.25, 30.5, 30.75, 31, 31.25, 31.5, 31.75, 32, 32.25, 32.5, 32.75, 33, 33.25, 33.5, 33.75, 34, 34.25, 34.5, 34.75, 35, 35.25, 35.5, 35.75, 36, 36.25, 36.5, 36.75, 37, 37.25, 37.5, 37.75, 38, 38.25, 38.5, 38.75, 39, 39.25, 39.5, 39.75, 40 ];
gen.Turbine.wind_turbine_powercurve_powerout =[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21.32, 33.509999999999998, 45.689999999999998, 65.209999999999994, 79.829999999999998, 104.25, 128.66, 157.97, 187.27000000000001, 216.58000000000001, 250.78, 292.31999999999999, 333.85000000000002, 375.39999999999998, 426.72000000000003, 475.60000000000002, 534.26999999999998, 597.80999999999995, 656.49000000000001, 724.94000000000005, 798.28999999999996, 871.63, 940.08000000000004, 1010, 1060, 1130, 1190, 1240, 1290, 1330, 1370, 1390, 1410, 1430, 1440, 1460, 1470, 1475, 1480, 1485, 1490, 1495, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
gen.Turbine.wind_turbine_hub_ht = 80
gen.Turbine.wind_turbine_max_cp = 0.45
gen.Farm.wind_farm_wake_model = 0
gen.Farm.wind_resource_turbulence_coeff = 0.1
turbine_kw = max(gen.Turbine.wind_turbine_powercurve_powerout)
n_turbines = int(desired_system_capacity // turbine_kw)
gen.Farm.system_capacity = turbine_kw * n_turbines
xcoords, ycoords = _set_num_turbines_in_row(n_turbines=n_turbines, spacing=250)
gen.Farm.wind_farm_xCoordinates = xcoords
gen.Farm.wind_farm_yCoordinates = ycoords
gen.Losses.wake_int_loss = 0
gen.Losses.wake_ext_loss = 1.1
gen.Losses.wake_future_loss = 0
gen.Losses.avail_bop_loss = 0.5
gen.Losses.avail_grid_loss = 1.5
gen.Losses.avail_turb_loss = 3.58
gen.Losses.elec_eff_loss = 1.91
gen.Losses.elec_parasitic_loss = 0.1
gen.Losses.env_degrad_loss = 1.8
gen.Losses.env_exposure_loss = 0
gen.Losses.env_env_loss = 0.4
gen.Losses.env_icing_loss = 0.21
gen.Losses.ops_env_loss = 1
gen.Losses.ops_grid_loss = 0.84
gen.Losses.ops_load_loss = 0.99
gen.Losses.ops_strategies_loss = 0
gen.Losses.turb_generic_loss = 1.7
gen.Losses.turb_hysteresis_loss = 0.4
gen.Losses.turb_perf_loss = 1.1
gen.Losses.turb_specific_loss = 0.81
gen.Uncertainty.total_uncert = 12.085
gen.execute()